In [0]:
# Path to the uploaded file
file_path = "foodly_company_documents.txt"

# Read the entire text
with open(file_path, "r") as f:
    policy_text = f.read()
    print(policy_text)


In [0]:
from langchain_community.document_loaders import TextLoader

file_path = "foodly_company_documents.txt"
# Load the document
loader = TextLoader(file_path, encoding="utf-8")
documents = loader.load()

print(f"Loaded {len(documents)} document(s).")

print(documents[0].metadata)
print(documents[0].page_content[:1000])


In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,      # ~800 characters per chunk
    chunk_overlap=100,   # 100 characters overlap to preserve context
)

docs = text_splitter.split_documents(documents)

print(f"Created {len(docs)} chunks.")



print(docs)

In [0]:
%sql

create catalog if not exists agents;
create schema if not exists agents.main;

In [0]:
# Prepare data for Delta
chunk_data = []
for i, d in enumerate(docs):
    chunk_data.append({
        "chunk_id": i + 1,
        "content": d.page_content,
        "metadata": str(d.metadata)  # store metadata as JSON-like string
    })

# Convert to dataframe

spark_df = spark.createDataFrame(chunk_data)

# Save as Delta
spark_df.write.format("delta").mode("overwrite").saveAsTable("agents.main.foodly_policy_chunks")

In [0]:
spark.sql("ALTER TABLE agents.main.foodly_policy_chunks SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

### Now you can create vector serach index using this table, Follow the below doc

https://docs.databricks.com/aws/en/generative-ai/create-query-vector-search